# Reduction of DFOSC data with CCDproc 

See here for the documentation of https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#clean-image 

In [1]:
# import the relevant packages 
import sys
import os
import numpy as np
  
from astropy.io import fits
from astropy import units as u

import ccdproc 
from ccdproc import CCDData

from ccdproc import ImageFileCollection

Define where are the raw and the reduced data? 

In [2]:
inputdir = '/Users/lasilla/data/2020-02-10/raw/'
outputdir = '/Users/lasilla/data/2020-02-10/reduced/NGC3311'

In [3]:
# if there's no output directory, make one: 
if not os.path.isdir(outputdir): 
    os.mkdir(outputdir)
os.chdir(outputdir)

Some properties of the DFOSC CCD

In [4]:
gain = 0.164 * u.electron / u.adu
ron = 4 * u.electron

We first want to sort the data according to its type (i.e. bias, flat...). The keyword in the fits header is `object`.

In [5]:
# collect the image files 
ifc = ImageFileCollection(inputdir, keywords='*')
ifc.files

['Borisov_test_000001.fits',
 'Borisov_test_000002.fits',
 'Borisov_test_000003.fits',
 'NGC2997_B_000001.fits',
 'NGC2997_B_000002.fits',
 'NGC2997_B_000003.fits',
 'NGC2997_B_000004.fits',
 'NGC2997_Halph_000001.fits',
 'NGC2997_Halph_000002.fits',
 'NGC2997_Halph_000003.fits',
 'NGC2997_Halph_000004.fits',
 'NGC2997_Halph_000005.fits',
 'NGC2997_I_000001.fits',
 'NGC2997_I_000002.fits',
 'NGC2997_I_000003.fits',
 'NGC2997_R_000001.fits',
 'NGC2997_R_000002.fits',
 'NGC2997_R_000003.fits',
 'NGC2997_V_000002.fits',
 'NGC2997_V_000003.fits',
 'NGC2997_V_000004.fits',
 'NGC2997_test_000001.fits',
 'NGC3265_test_000001.fits',
 'NGC3308_R_000001.fits',
 'NGC3308_R_000002.fits',
 'NGC3308_R_000003.fits',
 'NGC3308_V_000001.fits',
 'NGC3308_V_000002.fits',
 'NGC3308_V_000003.fits',
 'NGC3308_test_000001.fits',
 'NGC3308_test_000002.fits',
 'NGC3308_test_000003.fits',
 'NGC3311_B_000001.fits',
 'NGC3311_B_000002.fits',
 'NGC3311_B_000003.fits',
 'NGC3311_B_000004.fits',
 'NGC3311_B_000005.f

In [7]:
matches_bias = ifc.summary['object'] == 'bias'
matches_flat = ifc.summary['object'] == 'skyflat_I'
matches_sci_I = (ifc.summary['object'] == 'NGC3311_I') & (ifc.summary['filtb'] == 'I')
bias_files = [ifc.location + file for file in np.array(ifc.files)[matches_bias]]
flat_files_I = [ifc.location + file for file in np.array(ifc.files)[matches_flat]] 
science_files_I = [ifc.location + file for file in np.array(ifc.files)[matches_sci_I]]


In [8]:
bias_files

['/Users/lasilla/data/2020-02-10/raw/bias_000001.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000002.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000003.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000004.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000005.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000006.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000007.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000008.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000009.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000010.fits',
 '/Users/lasilla/data/2020-02-10/raw/bias_000011.fits']

In [9]:
matches_flat = ifc.summary['object'] == 'skyflat_V'
matches_sci_V = (ifc.summary['object'] == 'NGC3311_V') & (ifc.summary['filtb'] == 'V')
bias_files = [ifc.location + file for file in np.array(ifc.files)[matches_bias]]
flat_files_V = [ifc.location + file for file in np.array(ifc.files)[matches_flat]] 
science_files_V = [ifc.location + file for file in np.array(ifc.files)[matches_sci_V]]

In [10]:
matches_flat = ifc.summary['object'] == 'skyflat_R'
matches_sci_R = (ifc.summary['object'] == 'NGC3311_R') & (ifc.summary['filtb'] == 'R')
bias_files = [ifc.location + file for file in np.array(ifc.files)[matches_bias]]
flat_files_R = [ifc.location + file for file in np.array(ifc.files)[matches_flat]] 
science_files_R = [ifc.location + file for file in np.array(ifc.files)[matches_sci_R]]

In [11]:
matches_flat = ifc.summary['object'] == 'skyflat_B'
matches_sci_B = (ifc.summary['object'] == 'NGC3311_B') & (ifc.summary['filtb'] == 'B')
bias_files = [ifc.location + file for file in np.array(ifc.files)[matches_bias]]
flat_files_B = [ifc.location + file for file in np.array(ifc.files)[matches_flat]] 
science_files_B = [ifc.location + file for file in np.array(ifc.files)[matches_sci_B]]

In [12]:
matches_flat = ifc.summary['object'] == 'skyflat_Ha'
matches_sci_Ha = (ifc.summary['object'] == 'NGC3311_Halph') & (ifc.summary['filtb'] == 'H-alpha_rs')
bias_files = [ifc.location + file for file in np.array(ifc.files)[matches_bias]]
flat_files_Ha = [ifc.location + file for file in np.array(ifc.files)[matches_flat]] 
science_files_Ha = [ifc.location + file for file in np.array(ifc.files)[matches_sci_Ha]]

In [14]:
print(science_files_B)

['/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000001.fits', '/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000002.fits', '/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000003.fits', '/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000004.fits', '/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000005.fits']


We next create the master bias frame:

In [15]:
bias_list = []
for filename in bias_files:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    bias_list.append(ccd)
master_bias = ccdproc.combine(bias_list, method='median')
master_bias.write('master_bias.fits', clobber=True)

a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected.


/Users/lasilla/data/2020-02-10/raw/bias_000001.fits
/Users/lasilla/data/2020-02-10/raw/bias_000002.fits
/Users/lasilla/data/2020-02-10/raw/bias_000003.fits
/Users/lasilla/data/2020-02-10/raw/bias_000004.fits
/Users/lasilla/data/2020-02-10/raw/bias_000005.fits
/Users/lasilla/data/2020-02-10/raw/bias_000006.fits
/Users/lasilla/data/2020-02-10/raw/bias_000007.fits
/Users/lasilla/data/2020-02-10/raw/bias_000008.fits
/Users/lasilla/data/2020-02-10/raw/bias_000009.fits
/Users/lasilla/data/2020-02-10/raw/bias_000010.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/data/2020-02-10/raw/bias_000011.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


Now we create the master flat

In [16]:
flat_I_list = []
for filename in flat_files_I:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    exptime = ccd.header['EXPTIME']
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    flat_I_list.append(ccd)
master_flat_I = ccdproc.combine(flat_I_list, method='median')
master_flat_I.write('master_flat_I.fits', clobber=True)
# correct for the gain 
master_flat_I = ccdproc.gain_correct(master_flat_I, gain=gain)


/Users/lasilla/data/2020-02-10/raw/skyflat_I_000001.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000002.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000003.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000004.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000005.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000006.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000007.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000008.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000009.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_I_000010.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/data/2020-02-10/raw/skyflat_I_000011.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


In [17]:
flat_V_list = []
for filename in flat_files_V:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    exptime = ccd.header['EXPTIME']
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    flat_V_list.append(ccd)
master_flat_V = ccdproc.combine(flat_V_list, method='median')
master_flat_V.write('master_flat_V.fits', clobber=True)
# correct for the gain 
master_flat_V = ccdproc.gain_correct(master_flat_V, gain=gain)

/Users/lasilla/data/2020-02-10/raw/skyflat_V_000001.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_V_000002.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_V_000003.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/data/2020-02-10/raw/skyflat_V_000004.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


In [18]:
flat_R_list = []
for filename in flat_files_R[1:]:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    exptime = ccd.header['EXPTIME']
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    flat_R_list.append(ccd)
master_flat_R = ccdproc.combine(flat_R_list, method='median')
master_flat_R.write('master_flat_R.fits', clobber=True)
# correct for the gain 
master_flat_R = ccdproc.gain_correct(master_flat_R, gain=gain)

/Users/lasilla/data/2020-02-10/raw/skyflat_R_000002.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_R_000003.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_R_000004.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_R_000005.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/data/2020-02-10/raw/skyflat_R_000006.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


In [19]:
flat_B_list = []
for filename in flat_files_B:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    exptime = ccd.header['EXPTIME']
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    flat_B_list.append(ccd)
master_flat_B = ccdproc.combine(flat_B_list, method='median')
master_flat_B.write('master_flat_B.fits', clobber=True)
# correct for the gain 
master_flat_B = ccdproc.gain_correct(master_flat_B, gain=gain)

/Users/lasilla/data/2020-02-10/raw/skyflat_B_000001.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_B_000002.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_B_000003.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_B_000004.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_B_000005.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/data/2020-02-10/raw/skyflat_B_000006.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


In [20]:
flat_Ha_list = []
for filename in flat_files_Ha:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    exptime = ccd.header['EXPTIME']
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    flat_Ha_list.append(ccd)
master_flat_Ha = ccdproc.combine(flat_Ha_list, method='median')
master_flat_Ha.write('master_flat_Ha.fits', clobber=True)
# correct for the gain 
master_flat_Ha = ccdproc.gain_correct(master_flat_Ha, gain=gain)

/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000001.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000002.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000003.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000004.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000005.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000006.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000007.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000008.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000009.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000010.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000011.fits
/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000012.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/data/2020-02-10/raw/skyflat_Ha_000013.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


And now we just need to correct the science files :) We also remove cosmic rays^[I didn't see any in the raw data, but better save than sorry] and scale by the exposure time such that the resulting file is in counts/s.

In [21]:
for filename in science_files_I:
    print(filename)
    savename = os.path.basename(filename)
    hdu = fits.open(filename)
    ccd = CCDData(hdu['PRIMARY'].data, header=hdu['PRIMARY'].header, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    ccd = ccdproc.flat_correct(ccd, master_flat_I)
    ccd.write('sub_'+ savename, clobber=True)
    ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=5) # remove cosmic rays
    ccd.write('cr_'+ savename, clobber=True)
    # gain correct 
    ccd = ccdproc.gain_correct(ccd, gain=gain)
    # create variance map 
    ccd_var = ccdproc.create_deviation(ccd, readnoise=ron)
    # scale to 1s exposure time
    ccd = ccd.divide(ccd.header['EXPTIME'])
    ccd.write('obj_'+ savename, clobber=True)
    ccd_var.write('var_' + savename, clobber=True)

/Users/lasilla/opt/anaconda2/envs/photanalysis/lib/python2.7/site-packages/astropy/units/quantity.py:640: RuntimeWarning: divide by zero encountered in true_divide
  *arrays, **kwargs)


/Users/lasilla/data/2020-02-10/raw/NGC3311_I_000001.fits


/Users/lasilla/opt/anaconda2/envs/photanalysis/lib/python2.7/site-packages/ccdproc/core.py:1369: RuntimeWarning: invalid value encountered in less
  psfsize=psfsize, psfk=psfk, psfbeta=psfbeta, verbose=verbose)
/Users/lasilla/opt/anaconda2/envs/photanalysis/lib/python2.7/site-packages/ccdproc/core.py:1369: RuntimeWarning: invalid value encountered in subtract
  psfsize=psfsize, psfk=psfk, psfbeta=psfbeta, verbose=verbose)
/Users/lasilla/opt/anaconda2/envs/photanalysis/lib/python2.7/site-packages/ccdproc/core.py:1369: RuntimeWarning: invalid value encountered in greater
  psfsize=psfsize, psfk=psfk, psfbeta=psfbeta, verbose=verbose)
/Users/lasilla/opt/anaconda2/envs/photanalysis/lib/python2.7/site-packages/ccdproc/core.py:322: RuntimeWarning: invalid value encountered in sqrt
  var = (gain_value * ccd_data.data + readnoise_value ** 2) ** 0.5


/Users/lasilla/data/2020-02-10/raw/NGC3311_I_000002.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_I_000003.fits


Now the only thing left is do to the stacking. 

In [23]:
for filename in science_files_V:
    print(filename)
    savename = os.path.basename(filename)
    hdu = fits.open(filename)
    ccd = CCDData(hdu['PRIMARY'].data, header=hdu['PRIMARY'].header, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    ccd = ccdproc.flat_correct(ccd, master_flat_V)
    ccd.write('sub_'+ savename, clobber=True)
    ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=5) # remove cosmic rays
    ccd.write('cr_'+ savename, clobber=True)
    # gain correct 
    ccd = ccdproc.gain_correct(ccd, gain=gain)
    # create variance map 
    ccd_var = ccdproc.create_deviation(ccd, readnoise=ron)
    # scale to 1s exposure time
    ccd = ccd.divide(ccd.header['EXPTIME'])
    ccd.write('obj_'+ savename, clobber=True)
    ccd_var.write('var_' + savename, clobber=True)

/Users/lasilla/data/2020-02-10/raw/NGC3311_V_000001.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_V_000002.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_V_000003.fits


In [24]:
for filename in science_files_R:
    print(filename)
    savename = os.path.basename(filename)
    hdu = fits.open(filename)
    ccd = CCDData(hdu['PRIMARY'].data, header=hdu['PRIMARY'].header, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    ccd = ccdproc.flat_correct(ccd, master_flat_R)
    ccd.write('sub_'+ savename, clobber=True)
    ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=5) # remove cosmic rays
    ccd.write('cr_'+ savename, clobber=True)
    # gain correct 
    ccd = ccdproc.gain_correct(ccd, gain=gain)
    # create variance map 
    ccd_Rar = ccdproc.create_deviation(ccd, readnoise=ron)
    # scale to 1s exposure time
    ccd = ccd.divide(ccd.header['EXPTIME'])
    ccd.write('obj_'+ savename, clobber=True)
    ccd_Rar.write('var_' + savename, clobber=True)


/Users/lasilla/data/2020-02-10/raw/NGC3311_R_000001.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_R_000002.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_R_000003.fits


In [25]:
for filename in science_files_B:
    print(filename)
    savename = os.path.basename(filename)
    hdu = fits.open(filename)
    ccd = CCDData(hdu['PRIMARY'].data, header=hdu['PRIMARY'].header, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    ccd = ccdproc.flat_correct(ccd, master_flat_B)
    ccd.write('sub_'+ savename, clobber=True)
    ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=5) # remove cosmic rays
    ccd.write('cr_'+ savename, clobber=True)
    # gain correct 
    ccd = ccdproc.gain_correct(ccd, gain=gain)
    # create variance map 
    ccd_Rar = ccdproc.create_deviation(ccd, readnoise=ron)
    # scale to 1s exposure time
    ccd = ccd.divide(ccd.header['EXPTIME'])
    ccd.write('obj_'+ savename, clobber=True)
    ccd_Rar.write('var_' + savename, clobber=True)



/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000001.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000002.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000003.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000004.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_B_000005.fits


In [26]:
for filename in science_files_Ha:
    print(filename)
    savename = os.path.basename(filename)
    hdu = fits.open(filename)
    ccd = CCDData(hdu['PRIMARY'].data, header=hdu['PRIMARY'].header, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[5:71, :]')
    ccd = ccdproc.trim_image(ccd, fits_section='[79:2100, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    ccd = ccdproc.flat_correct(ccd, master_flat_Ha)
    ccd.write('sub_'+ savename, clobber=True)
    ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=5) # remove cosmic rays
    ccd.write('cr_'+ savename, clobber=True)
    # gain correct 
    ccd = ccdproc.gain_correct(ccd, gain=gain)
    # create variance map 
    ccd_Rar = ccdproc.create_deviation(ccd, readnoise=ron)
    # scale to 1s exposure time
    ccd = ccd.divide(ccd.header['EXPTIME'])
    ccd.write('obj_'+ savename, clobber=True)
    ccd_Rar.write('var_' + savename, clobber=True)



/Users/lasilla/data/2020-02-10/raw/NGC3311_Halph_000001.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_Halph_000002.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_Halph_000003.fits
/Users/lasilla/data/2020-02-10/raw/NGC3311_Halph_000004.fits
